In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('/content/drive/MyDrive/SENG 607/processedData_Dataminer.csv')
df.head()

,Unnamed: 0,Project,created_on,dependencies,desc,id,relationCount,status,subject,tracker
0,0,Redmine,2021-01-11T10:42:46Z,NaN,How do I export a list which includes the pare...,34583,0,New,Group by parent project,Feature
1,1,Redmine,2021-01-06T09:07:16Z,NaN,"Currently, it is possible to re-title an issue...",34566,0,New,Always trim title,Feature
2,2,Redmine,2021-01-05T09:09:54Z,34556relates8562#,"Currently, the ""Watchers"" section in the new i...",34556,1,New,Setting to change the maximum number to displa...,Feature
3,3,Redmine,2021-01-02T22:18:17Z,30459blocks34549#,Add keyboard shortcuts for the following butto...,34549,1,New,Add keyboard shortcuts for wiki toolbar buttons,Feature
4,4,Redmine,2020-12-23T11:20:28Z,NaN,At the moment we can assign Tickets to a group...,34496,0,New,Assign Tickets only to groups,Feature


In [3]:
df.drop(['Unnamed: 0','created_on','tracker','status'],axis=1,inplace=True)
df= df.dropna(subset=['desc']).reset_index(drop=True)

In [4]:
df.head()

,Project,dependencies,desc,id,relationCount,subject
0,Redmine,NaN,How do I export a list which includes the pare...,34583,0,Group by parent project
1,Redmine,NaN,"Currently, it is possible to re-title an issue...",34566,0,Always trim title
2,Redmine,34556relates8562#,"Currently, the ""Watchers"" section in the new i...",34556,1,Setting to change the maximum number to displa...
3,Redmine,30459blocks34549#,Add keyboard shortcuts for the following butto...,34549,1,Add keyboard shortcuts for wiki toolbar buttons
4,Redmine,NaN,At the moment we can assign Tickets to a group...,34496,0,Assign Tickets only to groups


In [5]:
df1= df.dropna(subset=['dependencies']).reset_index(drop=True)

In [6]:
df1.head()

,Project,dependencies,desc,id,relationCount,subject
0,Redmine,34556relates8562#,"Currently, the ""Watchers"" section in the new i...",34556,1,Setting to change the maximum number to displa...
1,Redmine,30459blocks34549#,Add keyboard shortcuts for the following butto...,34549,1,Add keyboard shortcuts for wiki toolbar buttons
2,Redmine,34495relates1237#,"""Two-factor authentication"" on a new user form...",34495,1,"Don't show ""Two-factor authentication"" on new ..."
3,Redmine,34340relates6938#,All projects are displayed in the tracker edit...,34340,1,Make archived projects visually distinguishabl...
4,Redmine,33383relates34337#,#33383#note-3:\r\n> Because the update can bre...,34337,1,Remove jQuery Migrate


In [7]:
df1.isnull().sum()

Project          0
dependencies     0
desc             0
id               0
relationCount    0
subject          0
dtype: int64

In [8]:
df1.shape

(3401, 6)

In [9]:
import re
id1 = []
id2 = []
desc1 = []
desc2 = []
dependency  = []

for i in range(3401):
  a = df1['dependencies'][i]
  a = a.split('#')
  for j in range(len(a)-1):
    b = a[j]
    temp = re.findall(r'\d+', b)                         # For extracting id's
    r = list(map(int, temp))
    id1.append(r[0])
    id2.append(r[1])

    c = df1[df1['id']==r[0]]['desc'].to_string()          # For extracting description wrt to each id
    c1 = df1[df1['id']==r[1]]['desc'].to_string() 
    desc1.append(c[5:])
    desc2.append(c1[5:])

    temp1 = str(r[0])
    temp2 = str(r[1])
    result = re.search(f'{temp1}(.*){temp2}',b).group(1) # For extracting type of dependency
    dependency.append(result)

In [10]:
df2 = pd.DataFrame({'id1':id1,'Description1':desc1,'id2':id2,'Description2':desc2,'Type of dependency':dependency})
df2.head(10)

,id1,Description1,id2,Description2,Type of dependency
0,34556,"Currently, the ""Watchers"" section in the new i...",8562,We face the problem that we have many proje...,relates
1,30459,"In Redmine 3.4 and earlier, you could update...",34549,Add keyboard shortcuts for the following butto...,blocks
2,34495,"""Two-factor authentication"" on a new user form...",1237,Please add support for a one time password ...,relates
3,34340,All projects are displayed in the tracker edit...,6938,I have a few ideas to enhance the edit trac...,relates
4,33383,jQuery 2 is end of life and we should update ...,34337,#33383#note-3:\r\n> Because the update can bre...,relates
5,34336,"Currently, there is no way to see the latest p...",12664,"s([], )",relates
6,34258,"When there are many custom fields or projects,...",9258,Allow creating a new role by copying an exi...,relates
7,34241,The upcoming Redmine 4.2 has a two-factor auth...,1237,Please add support for a one time password ...,relates
8,34142,The official support of Ruby 2.3 has already e...,32530,"s([], )",relates
9,27780,"s([], )",34142,The official support of Ruby 2.3 has already e...,relates


In [11]:
# Deleting empty lists
l1 = []
l2 = []
for i in range(6810):
  if df2['Description1'][i] == 's([], )':
    l1.append(i)
  if df2['Description2'][i] == 's([], )':
    l2.append(i)
l = list(set(l1+l2))

In [12]:
df2.drop(l,inplace = True)

In [13]:
df2 = df2.reset_index(drop = True)

**Final DataFrame**

In [14]:
df2.head(10)

,id1,Description1,id2,Description2,Type of dependency
0,34556,"Currently, the ""Watchers"" section in the new i...",8562,We face the problem that we have many proje...,relates
1,30459,"In Redmine 3.4 and earlier, you could update...",34549,Add keyboard shortcuts for the following butto...,blocks
2,34495,"""Two-factor authentication"" on a new user form...",1237,Please add support for a one time password ...,relates
3,34340,All projects are displayed in the tracker edit...,6938,I have a few ideas to enhance the edit trac...,relates
4,33383,jQuery 2 is end of life and we should update ...,34337,#33383#note-3:\r\n> Because the update can bre...,relates
5,34258,"When there are many custom fields or projects,...",9258,Allow creating a new role by copying an exi...,relates
6,34241,The upcoming Redmine 4.2 has a two-factor auth...,1237,Please add support for a one time password ...,relates
7,29473,"While editing issue, Ctrl+Enter should invok...",33918,This feature should be documented on [[Redmine...,precedes
8,33884,"Issue details page shows ""Due in X days"" info...",31499,"This patch adds ""Due in X days"" / ""X days lat...",relates
9,33834,This patch adds open/closed badge to email no...,33254,#29391 added a useful badge that shows the st...,relates


In [15]:
y1,y2 = [],[]
y = df2['Description1']
z = df2['Description2']
for i in range(len(y)):
  q = y[i].split(' ')
  if len(q) <2:
    y1.append(i)
for i in range(len(y)):
  q1 = z[i].split(' ')
  if len(q1) <2:
    y2.append(i)
print(len(y1))
print(len(y2))
#q2 = list(set(y1+y2))
#df2.drop(q2,inplace=True)
#df2 = df2.reset_index(drop=True)

0
0


In [16]:
df2.shape

(5224, 5)

In [17]:
df2['Type of dependency'].value_counts()

relates       2824
duplicates    2326
blocks          36
copied_to       20
precedes        18
Name: Type of dependency, dtype: int64

In [18]:
df2.isnull().sum()

id1                   0
Description1          0
id2                   0
Description2          0
Type of dependency    0
dtype: int64

In [21]:
#df2.to_csv('/content/drive/MyDrive/SENG 607/Redmine_Pos_modified.csv')

**Creating independent pairs**

In [29]:
df['relationCount'].value_counts()

0     3418
1     2110
2      607
3      289
4      143
5       76
6       41
7       29
8       27
9       18
10      16
13      10
11       9
12       9
16       3
17       3
28       2
24       2
27       1
23       1
14       1
18       1
22       1
15       1
21       1
Name: relationCount, dtype: int64

In [30]:
i1 = []
i2 = []
d1 = []
d2 = []

n = df[df['relationCount']==0]['id'].values

for i in range(10000):
  d = np.random.choice(n,size=2)
  i1.append(d[0])
  i2.append(d[1])

  c = df[df['id']==d[0]]['desc'].to_string()          # For extracting description wrt to each id
  c1 = df[df['id']==d[1]]['desc'].to_string()
  d1.append(c[5:])
  d2.append(c1[5:])

In [31]:
def ext(x):
  return 'independent'

In [32]:
df3 = pd.DataFrame({'id1':i1,'Description1':d1,'id2':i2,'Description2':d2})
df3.head()

,id1,Description1,id2,Description2
0,1497,"Add ability to add ""Target Version"" as a fi...",278,"Hi,\r\n\r\nCan you add the option to add a ..."
1,24188,Can I display the responsible name on gantt?...,3796,"Dear All,\r\n\r\nUsers can create new issue..."
2,10523,"Hey guys,\r\n\r\nI really need an additiona...",317,I created an issue with lots of newlines. I...
3,12888,Since we've started using Redmine 2.2.x we'...,32695,The subject says it all... Please add the 4....
4,12559,I have a project set up in redmine. I want ...,4013,"Hello, \r\nI usually use sub-projects. \r\n..."


In [34]:
df3['Type of dependency'] = df3['id1'].apply(ext)

In [35]:
df3.head()

,id1,Description1,id2,Description2,Type of dependency
0,1497,"Add ability to add ""Target Version"" as a fi...",278,"Hi,\r\n\r\nCan you add the option to add a ...",independent
1,24188,Can I display the responsible name on gantt?...,3796,"Dear All,\r\n\r\nUsers can create new issue...",independent
2,10523,"Hey guys,\r\n\r\nI really need an additiona...",317,I created an issue with lots of newlines. I...,independent
3,12888,Since we've started using Redmine 2.2.x we'...,32695,The subject says it all... Please add the 4....,independent
4,12559,I have a project set up in redmine. I want ...,4013,"Hello, \r\nI usually use sub-projects. \r\n...",independent


In [36]:
# Sentence Length Analysis
y1,y2 = [],[]
y = df3['Description1']
z = df3['Description2']
for i in range(len(y)):
  q = y[i].split(' ')
  if len(q) <2:
    y1.append(i)
for i in range(len(y)):
  q1 = z[i].split(' ')
  if len(q1) <2:
    y2.append(i)
print(len(y1))
print(len(y2))
q2 = list(set(y1+y2))
df3.drop(q2,inplace=True)
df3 = df3.reset_index(drop=True)

0
0


In [37]:
df3.head()

,id1,Description1,id2,Description2,Type of dependency
0,1497,"Add ability to add ""Target Version"" as a fi...",278,"Hi,\r\n\r\nCan you add the option to add a ...",independent
1,24188,Can I display the responsible name on gantt?...,3796,"Dear All,\r\n\r\nUsers can create new issue...",independent
2,10523,"Hey guys,\r\n\r\nI really need an additiona...",317,I created an issue with lots of newlines. I...,independent
3,12888,Since we've started using Redmine 2.2.x we'...,32695,The subject says it all... Please add the 4....,independent
4,12559,I have a project set up in redmine. I want ...,4013,"Hello, \r\nI usually use sub-projects. \r\n...",independent


In [38]:
df3.isnull().sum()

id1                   0
Description1          0
id2                   0
Description2          0
Type of dependency    0
dtype: int64

In [39]:
#df3.to_csv('/content/drive/MyDrive/SENG 607/Redmine_Neg_modified.csv')

**Processed Dependent Data**

In [ ]:
fd = pd.read_csv('/content/drive/MyDrive/SENG 607/AL_redmineP.csv')

In [ ]:
fd.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
fd.head()

,id1,Description1,id2,Description2,Type of dependency
0,34556,currently watcher section new,8562,face problem many project,relates
1,30459,redmine earlier could update,34549,add keyboard shortcut following butto,blocks
2,34495,twofactor authentication new user form,1237,please add support one time password ser,relates
3,34340,project displayed tracker edit,6938,idea enhance edit tracker,relates
4,33383,jquery end life update,34337,update bre,relates


In [ ]:
fd.shape

(5224, 5)

In [ ]:
fd.isnull().sum()

id1                    0
Description1          22
id2                    0
Description2          14
Type of dependency     0
dtype: int64

In [ ]:
fd.dropna(inplace=True)
fd = fd.reset_index(drop=True)

In [ ]:
y1,y2 = [],[]
y = fd['Description1']
z = fd['Description2']
for i in range(len(y)):
  if len(str(y[i])) < 5 :
    y1.append(i)
for i in range(len(z)):
  if len(str(z[i])) < 5 :
    y2.append(i)
print(len(y1))
print(len(y2))
r = list(set(y1+y2))

0
0


In [ ]:
fd.shape

(5190, 5)

In [ ]:
a = fd[fd['id2']==3713]['Description2'].index
b = fd[fd['id2']==1255]['Description2'].index
a = list(a)
b = list(b)
c = a+b
fd.drop(c,inplace=True)
fd = fd.reset_index(drop=True)

In [ ]:
fd.head()

,id1,Description1,id2,Description2,Type of dependency
0,34556,currently watcher section new,8562,face problem many project,relates
1,30459,redmine earlier could update,34549,add keyboard shortcut following butto,blocks
2,34495,twofactor authentication new user form,1237,please add support one time password ser,relates
3,34340,project displayed tracker edit,6938,idea enhance edit tracker,relates
4,33383,jquery end life update,34337,update bre,relates


In [ ]:
fd.shape

(5186, 5)

In [ ]:
fd.isnull().sum()

id1                   0
Description1          0
id2                   0
Description2          0
Type of dependency    0
dtype: int64

In [ ]:
fd['Type of dependency'].value_counts()

relates       2804
duplicates    2310
blocks          36
copied_to       18
precedes        18
Name: Type of dependency, dtype: int64

In [ ]:
#fd.to_csv('/content/drive/MyDrive/Redmine_Processed_ALPos.csv')

**Independent pairs processed data**

In [ ]:
fd1 = pd.read_csv('/content/drive/MyDrive/SENG 607/AL_redmine_IndP.csv')

In [ ]:
fd1.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
fd1.head()

,id1,Description1,id2,Description2,Type of dependency
0,3314,editing user detail via admin panel,544,issue summary page get summar,independent
1,27315,sort column attachment alphabetical,12628,would also great enhancement,independent
2,27211,import field missing project let kno,6268,issue easy way get,independent
3,547,wiki extremely practical documentat,4695,redmine could provide special stack trace form,independent
4,4717,observed rndirect link,18122,installation tutorial step regarding,independent


In [ ]:
y1,y2 = [],[]
y = fd1['Description1']
z = fd1['Description2']
for i in range(len(y)):
  if len(str(y[i])) < 6 :
    y1.append(i)
for i in range(len(z)):
  if len(str(z[i])) < 6 :
    y2.append(i)
print(len(y1))
print(len(y2))
r = list(set(y1+y2))

346
334


In [ ]:
fd1.drop(r,inplace=True)
fd1 = fd1.reset_index(drop=True)

In [ ]:
fd1.head()

,id1,Description1,id2,Description2,Type of dependency
0,3314,editing user detail via admin panel,544,issue summary page get summar,independent
1,27315,sort column attachment alphabetical,12628,would also great enhancement,independent
2,27211,import field missing project let kno,6268,issue easy way get,independent
3,547,wiki extremely practical documentat,4695,redmine could provide special stack trace form,independent
4,4717,observed rndirect link,18122,installation tutorial step regarding,independent


In [ ]:
u,u1 = [],[]
drop1 = [9152,6660,5843,5842,5848,2886,7897,4553,9153,6516,2338,8665,78662,7673,13170,8402,5841,4937,6106]
drop2 = [7196,6160,7862,2886,9251,4553,8752,8791,806,7869,6106,5377,8635,8791,13787,8260,8752,1255,342,9251]
drop = drop1+drop2
for i in drop:
  a = fd1[fd1['id2']==i]['Description2'].index
  a = list(a)
  for j in a:
    u.append(j)
print(len(u))

drop2 = [7196,6160,7862,2886,9251,4553,8752,8791,806,7869,6106,5377,8635,8791,13787,8260,8752,1255,342,9251]
for i in drop:
  b = fd1[fd1['id1']==i]['Description1'].index
  b = list(b)
  for j in b:
    u1.append(j)
print(len(u1))
s = list(set(u1+u))
print(len(s))

543
521
863


In [ ]:
fd1.drop(s,inplace=True)
fd1 = fd1.reset_index(drop=True)

In [ ]:
fd1.head()

,id1,Description1,id2,Description2,Type of dependency
0,3314,editing user detail via admin panel,544,issue summary page get summar,independent
1,27315,sort column attachment alphabetical,12628,would also great enhancement,independent
2,27211,import field missing project let kno,6268,issue easy way get,independent
3,547,wiki extremely practical documentat,4695,redmine could provide special stack trace form,independent
4,4717,observed rndirect link,18122,installation tutorial step regarding,independent


In [ ]:
fd1.isnull().sum()

id1                   0
Description1          0
id2                   0
Description2          0
Type of dependency    0
dtype: int64

In [ ]:
#fd1.to_csv('/content/drive/MyDrive/Redmine_Processed_ALNeg.csv')

**BERT DATASET**

In [50]:
ber = pd.read_csv('/content/drive/MyDrive/SENG 607/Redmine_Processed_BERTPos_init_modified.csv')

In [51]:
ber.head()

,Unnamed: 0,id1,Description1,id2,Description2,Type of dependency
0,0,34556,currently the watchers section in the new i,8562,we face the problem that we have many project,relates
1,1,30459,in redmine and earlier you could update t,34549,add keyboard shortcuts for the following butto,blocks
2,2,34495,twofactor authentication on new user form,1237,please add support for one time password ser,relates
3,3,34340,all projects are displayed in the tracker edit,6938,have few ideas to enhance the edit tracker,relates
4,4,33383,jquery is end of life and we should update t,34337,because the update can bre,relates


In [52]:
ber.isnull().sum()

Unnamed: 0             0
id1                    0
Description1          20
id2                    0
Description2          14
Type of dependency     0
dtype: int64

In [53]:
ber.drop(['Unnamed: 0'],axis=1,inplace=True)

In [54]:
ber.dropna(inplace=True)
ber = ber.reset_index(drop=True)

In [55]:
ber.head()

,id1,Description1,id2,Description2,Type of dependency
0,34556,currently the watchers section in the new i,8562,we face the problem that we have many project,relates
1,30459,in redmine and earlier you could update t,34549,add keyboard shortcuts for the following butto,blocks
2,34495,twofactor authentication on new user form,1237,please add support for one time password ser,relates
3,34340,all projects are displayed in the tracker edit,6938,have few ideas to enhance the edit tracker,relates
4,33383,jquery is end of life and we should update t,34337,because the update can bre,relates


In [56]:
ber.isnull().sum()

id1                   0
Description1          0
id2                   0
Description2          0
Type of dependency    0
dtype: int64

In [57]:
def isEnglish(s):
  return s.isascii()

a1 , a2 = [] , []
a = ber['Description1'].values
b = ber['Description2'].values

for i in range(len(a)):
  if isEnglish(a[i])==False:
    a1.append(i)
for j in range(len(b)):
  if isEnglish(b[j])==False:
    a2.append(j)

print(len(a1))
print(len(a2))
a3 = list(set(a1+a2))
print(len(a3))

ber.drop(a3,inplace=True)
ber = ber.reset_index(drop=True)

6
16
22


In [58]:
# Sentence Length Analysis
y1,y2 = [],[]
y = ber['Description1']
z = ber['Description2']
for i in range(len(y)):
  q = y[i].split(' ')
  if len(q) <2:
    y1.append(i)
for i in range(len(y)):
  q1 = z[i].split(' ')
  if len(q1) <2:
    y2.append(i)
print(len(y1))
print(len(y2))
q2 = list(set(y1+y2))
ber.drop(q2,inplace=True)
ber = ber.reset_index(drop=True)

8
20


In [59]:
#ber.to_csv('/content/drive/MyDrive/SENG 607/Redmine_Processed_BERTPos_modified.csv')

**BERT independent**

In [72]:
beri = pd.read_csv('/content/drive/MyDrive/SENG 607/Redmine_Processed_BERTNeg_init_modified.csv')

In [73]:
beri.drop(['Unnamed: 0'],axis=1,inplace=True)

In [74]:
beri.head()

,id1,Description1,id2,Description2,Type of dependency
0,1497,add ability to add target version as filte,278,hirnrncan you add the option to add wik,independent
1,24188,can display the responsible name on ganttr,3796,dear allrnrnusers can create new issues d,independent
2,10523,hey guysrnrni really need an additional f,317,created an issue with lots of newlines sa,independent
3,12888,since weve started using redmine weve,32695,the subject says it all please add the,independent
4,12559,have project set up in redmine want to,4013,hello rni usually use subprojects rnwhe,independent


In [75]:
beri.shape

(10000, 5)

In [76]:
beri.isnull().sum()

id1                    0
Description1          34
id2                    0
Description2          31
Type of dependency     0
dtype: int64

In [77]:
beri.dropna(inplace=True)
beri = beri.reset_index(drop=True)

In [78]:
beri.head()

,id1,Description1,id2,Description2,Type of dependency
0,1497,add ability to add target version as filte,278,hirnrncan you add the option to add wik,independent
1,24188,can display the responsible name on ganttr,3796,dear allrnrnusers can create new issues d,independent
2,10523,hey guysrnrni really need an additional f,317,created an issue with lots of newlines sa,independent
3,12888,since weve started using redmine weve,32695,the subject says it all please add the,independent
4,12559,have project set up in redmine want to,4013,hello rni usually use subprojects rnwhe,independent


In [79]:
beri.isnull().sum()

id1                   0
Description1          0
id2                   0
Description2          0
Type of dependency    0
dtype: int64

In [80]:
beri.shape

(9935, 5)

In [81]:
def isEnglish(s):
  return s.isascii()

a1 , a2 = [] , []
a = beri['Description1'].values
b = beri['Description2'].values

for i in range(len(a)):
  if isEnglish(a[i])==False:
    a1.append(i)
for j in range(len(b)):
  if isEnglish(b[j])==False:
    a2.append(j)

print(len(a1))
print(len(a2))
a3 = list(set(a1+a2))
print(len(a3))

beri.drop(a3,inplace=True)
beri = beri.reset_index(drop=True)

153
181
332


In [82]:
# Sentence Length Analysis
y1,y2 = [],[]
y = beri['Description1']
z = beri['Description2']
for i in range(len(y)):
  q = y[i].split(' ')
  if len(q) <2:
    y1.append(i)
for i in range(len(y)):
  q1 = z[i].split(' ')
  if len(q1) <2:
    y2.append(i)
print(len(y1))
print(len(y2))
q2 = list(set(y1+y2))
beri.drop(q2,inplace=True)
beri = beri.reset_index(drop=True)

101
110


In [86]:
beri.isnull().sum()

id1                   0
Description1          0
id2                   0
Description2          0
Type of dependency    0
dtype: int64

In [87]:
beri.shape

(9393, 5)

In [88]:
#beri.to_csv('/content/drive/MyDrive/SENG 607/Redmine_Processed_BERTNeg_modified.csv')